In [1]:
import random

import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error

import tensorflow as tf
from tensorflow.keras.layers import Concatenate, Dense, Dot, Dropout, Embedding, Input, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import Callback, ModelCheckpoint

# I use Tensorflow 2.x here so that the Keras is included as part of the Tensorflow
# If you install the Keras aloneside you can just use the following imports

# from keras.layers import Concatenate, Dense, Dot, Dropout, Embedding, Input, Reshape
# from keras.models import Model
# from keras.callbacks import Callback, ModelCheckpoint

In [2]:
random.seed(2021)
np.random.seed(2021)
tf.random.set_seed(2021)

## Define the evaluation metric

In [3]:
from math import sqrt

def rmse(pred, actual):
    # Ignore ratings with value zero.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return sqrt(mean_squared_error(pred, actual))

## Data processing

In [4]:
train_df = pd.read_csv("data/train.csv")
valid_df = pd.read_csv("data/valid.csv")
train_df.head()

,user_id,business_id,stars
0,ec8f38aa91755dcf5837020d022ad384,ecaa90564e18dca1c7b653038f71d6bf,1.0
1,64fe4dd0a489c9b96a3e8d7fbd337888,ef118bb0ae1fc369e1f47d1b34f6acee,5.0
2,a49909b39426ebb3538aa837b5b88840,e8b182a923810d52981aa02d56dde799,5.0
3,a56726d5676d647e42e2aca54f21b075,250040e979eae9ef5912aa5a1d285e4e,5.0
4,3e19d8260e655ba87bea0922bac92266,e02880faf4d42fe1df7bd370fb1c787b,4.0


In [5]:
#Get the set of all user ids and set of all business ids in train set
user_set = set(train_df.user_id.unique())
business_set = set(train_df.business_id.unique())

#Build user vocabulary
user_vocab = dict(zip(user_set, range(1, len(user_set) + 1)))
#reserve the first row of the embedding matrix for users unseen in the training set
user_vocab['unk'] = 0
n_users = len(user_vocab)

#Build business vocabulary
business_vocab = dict(zip(business_set, range(1, len(business_set) + 1)))
#reserve the first row of the embedding matrix for businesses unseen in the training set
business_vocab['unk'] = 0
n_items = len(business_vocab)

print(n_users, n_items)

4981 10845


In [6]:
train_users = train_df.user_id.apply(lambda x: user_vocab[x]).values
train_items = train_df.business_id.apply(lambda x: business_vocab[x]).values
valid_users = valid_df.user_id.apply(lambda x: user_vocab[x] if x in user_vocab else 0).values
valid_items = valid_df.business_id.apply(lambda x: business_vocab[x] if x in business_vocab else 0).values

In [7]:
train_ratings = train_df.stars.values
valid_ratings = valid_df.stars.values

In [8]:
print(np.min(train_ratings), np.max(train_ratings))

1.0 5.0


## User/Item CF with `scipy` sparse matrix

In [9]:
from scipy.sparse import coo_matrix

In [10]:
train_matrix_sparse = coo_matrix((train_ratings, (train_users, train_items)))
print(train_matrix_sparse)

train_matrix = train_matrix_sparse.toarray()
print(train_matrix.shape)
print(train_matrix.sum()/n_users)
print(train_matrix.sum()/n_items)

  (2721, 4836)	1.0
  (3755, 3990)	5.0
  (306, 7858)	5.0
  (2600, 8050)	5.0
  (3729, 6251)	4.0
  (3565, 539)	5.0
  (4209, 1232)	4.0
  (1774, 1798)	2.0
  (2135, 4902)	5.0
  (1802, 3572)	5.0
  (4692, 4967)	3.0
  (1010, 8620)	5.0
  (1545, 518)	5.0
  (2577, 5549)	5.0
  (4617, 3073)	5.0
  (1717, 1130)	4.0
  (3699, 9809)	4.0
  (2580, 3902)	5.0
  (1793, 2147)	5.0
  (4692, 3671)	5.0
  (1147, 10625)	5.0
  (642, 6342)	2.0
  (258, 8506)	4.0
  (4740, 3915)	4.0
  (305, 5287)	3.0
  :	:
  (4627, 3040)	3.0
  (2004, 10796)	4.0
  (4671, 5555)	4.0
  (3026, 10042)	5.0
  (1819, 10705)	4.0
  (3028, 10394)	2.0
  (1063, 9651)	4.0
  (476, 3114)	3.0
  (899, 10272)	4.0
  (320, 1196)	3.0
  (3434, 3106)	5.0
  (1557, 1631)	4.0
  (1399, 3732)	2.0
  (663, 1980)	5.0
  (529, 2535)	3.0
  (464, 6652)	5.0
  (68, 2649)	5.0
  (1335, 5955)	5.0
  (4284, 4614)	5.0
  (3242, 8757)	5.0
  (2322, 2136)	5.0
  (3645, 5952)	4.0
  (2159, 2822)	4.0
  (274, 7588)	4.0
  (122, 9793)	5.0
(4981, 10845)
77.18530415579201
35.45043798985708


### User CF v.s. Item CF

In [34]:
from sklearn.metrics.pairwise import cosine_similarity

In [41]:
from numpy.linalg import norm

In [45]:
from tqdm import trange

In [46]:
def cosine_similarity_v2(X, Y):
    m, d1 = X.shape
    n, d2 = Y.shape
    ans = np.zeros((m,n))
    assert d1 == d2
    for i in trange(n):
        for j in range(m):
            valid_ids = (X[i] > 0) & (Y[j] > 0)
            x = X[i, valid_ids]
            y = Y[j, valid_ids]
            ans[i,j] = x.dot(y) / (norm(x) * norm(y)+1e-6)
    return ans
            

In [47]:
user_sim = cosine_similarity_v2(train_matrix, train_matrix)

print(user_sim.shape)
print(user_sim)

  4%|▍         | 203/4981 [00:30<12:07,  6.56it/s]


KeyboardInterrupt: 

In [12]:
user_norm_train_matrix = train_matrix - train_matrix.mean(axis=1, keepdims=True)
user_sim = cosine_similarity(user_norm_train_matrix, user_norm_train_matrix)

print(user_sim.shape)
print(user_sim)

(4981, 4981)
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00 -1.86236136e-03 ... -1.20710945e-03
  -7.34438352e-04 -1.55524801e-03]
 [ 0.00000000e+00 -1.86236136e-03  1.00000000e+00 ...  2.33066014e-02
  -2.54405630e-03 -5.38729834e-03]
 ...
 [ 0.00000000e+00 -1.20710945e-03  2.33066014e-02 ...  1.00000000e+00
  -1.64895732e-03 -3.49183508e-03]
 [ 0.00000000e+00 -7.34438352e-04 -2.54405630e-03 ... -1.64895732e-03
   1.00000000e+00 -2.12452781e-03]
 [ 0.00000000e+00 -1.55524801e-03 -5.38729834e-03 ... -3.49183508e-03
  -2.12452781e-03  1.00000000e+00]]


In [13]:
item_norm_train_matrix = train_matrix - train_matrix.mean(axis=0, keepdims=True)
item_sim = cosine_similarity(item_norm_train_matrix.T, item_norm_train_matrix.T)

print(item_sim.shape)
print(item_sim)

(10845, 10845)
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00 -1.29374180e-03 ... -1.10092635e-03
  -1.62468321e-03 -8.13712776e-04]
 [ 0.00000000e+00 -1.29374180e-03  1.00000000e+00 ... -2.09797158e-03
   4.41126030e-02 -1.55064530e-03]
 ...
 [ 0.00000000e+00 -1.10092635e-03 -2.09797158e-03 ...  1.00000000e+00
  -2.63463635e-03 -1.31954171e-03]
 [ 0.00000000e+00 -1.62468321e-03  4.41126030e-02 ... -2.63463635e-03
   1.00000000e+00 -1.94730306e-03]
 [ 0.00000000e+00 -8.13712776e-04 -1.55064530e-03 ... -1.31954171e-03
  -1.94730306e-03  1.00000000e+00]]


In [14]:
def user_cf_prediction(valid_users, valid_items, k, verbose=False):
    
    pred_ratings = []
    for u, i in zip(valid_users, valid_items):
        # get the candidate users that have boughted the target item i
        candidate_user_index = train_matrix[:, i] > 0
        if len(candidate_user_index) < 1:
            print(f"item {i} is not found in the training set")
            pred_ratings.append(train_matrix[u].mean())
            continue

        # compute the similarity of those users
        this_user_sim = user_sim[u].copy()
        this_user_sim[~candidate_user_index] = 0
        candidate_sim = user_sim[u, candidate_user_index]

        if verbose:
            print(f"predict rating for user_id {u} and item_id {i}")
            print(f"candidate user index is a boolean array {candidate_user_index}")
            print(f"there are {candidate_sim.shape[0]} candidate users")

        # select top k similar users with best absolute correlations
        topk_sim_users = np.abs(this_user_sim).argsort()[-k:][::-1]
        topk_sim = user_sim[u, topk_sim_users]
        topk_ratings = user_norm_train_matrix[topk_sim_users, i]
        if verbose:
            print(f"top {k} (if possible) similar users {topk_sim_users} are retrived")
            print(f"\t with similarity {topk_sim}")
            print(f"\t and ratings of {topk_ratings}")

        # make redictions by the user average and relative performance
        user_ave = train_matrix[u].mean()
        topk_ave = np.sum((topk_ratings * topk_sim))/(topk_sim.sum()+1e-6)
        pred_rating = user_ave + topk_ave
        pred_ratings.append(pred_rating)
        if verbose:
            print(f"predict the rating {pred_rating:.4f} = user average {user_ave:.4f} + topk sim average {topk_ave:.4f}")
            print()
    return np.asarray(pred_ratings)

In [15]:
pred_ratings = user_cf_prediction(valid_users[:5], valid_items[:5], 3, verbose=True)
print(pred_ratings, valid_ratings[:5])

predict rating for user_id 4751 and item_id 9671
candidate user index is a boolean array [False False False ... False False False]
there are 4 candidate users
top 3 (if possible) similar users [4842 3993 4435] are retrived
	 with similarity [ 0.05599403  0.01080387 -0.00166243]
	 and ratings of [4.9934532  2.93748271 4.99419087]
predict the rating 4.6606 = user average 0.0082 + topk sim average 4.6523

predict rating for user_id 2104 and item_id 3716
candidate user index is a boolean array [False False False ... False False False]
there are 24 candidate users
top 3 (if possible) similar users [4295 2284 2123] are retrived
	 with similarity [0.13758703 0.11640039 0.10527415]
	 and ratings of [3.98939604 4.98819733 3.96864915]
predict the rating 4.3139 = user average 0.0070 + topk sim average 4.3069

predict rating for user_id 4207 and item_id 6856
candidate user index is a boolean array [False False False ... False False False]
there are 18 candidate users
top 3 (if possible) similar us

In [16]:
pred_ratings = user_cf_prediction(valid_users[:5], valid_items[:5], 10, verbose=True)
print(pred_ratings, valid_ratings[:5])

predict rating for user_id 4751 and item_id 9671
candidate user index is a boolean array [False False False ... False False False]
there are 4 candidate users
top 10 (if possible) similar users [4842 3993 4435  729 1637 1655 1657 1658 1659 1660] are retrived
	 with similarity [ 0.05599403  0.01080387 -0.00166243 -0.00103704  0.13346756 -0.00096937
 -0.00110126 -0.00072386 -0.00103381 -0.00150919]
	 and ratings of [ 4.99345320e+00  2.93748271e+00  4.99419087e+00  2.99778700e+00
 -8.11433840e-03 -2.12079299e-03 -2.85846012e-03 -1.19870908e-03
 -2.30520977e-03 -4.51821116e-03]
predict the rating 1.5629 = user average 0.0082 + topk sim average 1.5547

predict rating for user_id 2104 and item_id 3716
candidate user index is a boolean array [False False False ... False False False]
there are 24 candidate users
top 10 (if possible) similar users [4295 2284 2123 3772 2731 1207  637 3978 1936 4046] are retrived
	 with similarity [0.13758703 0.11640039 0.10527415 0.09980196 0.09785796 0.08675663

In [17]:
for k in [1, 2, 3, 4, 5, 10]:
    pred_ratings = user_cf_prediction(valid_users, valid_items, k)
    score = rmse(pred_ratings, valid_ratings)
    print(f"user CF with top-{k} similar users, RMSE = {score}")

user CF with top-1 similar users, RMSE = 1.5069515049286584
user CF with top-2 similar users, RMSE = 1.6429612322007046
user CF with top-3 similar users, RMSE = 2.384089999944481
user CF with top-4 similar users, RMSE = 4.3451446065057855
user CF with top-5 similar users, RMSE = 4.330470484576127
user CF with top-10 similar users, RMSE = 8.796787091753346


In [18]:
def item_cf_prediction(valid_users, valid_items, k, verbose=False):
    
    pred_ratings = []
    for u, i in zip(valid_users, valid_items):
        candidate_item_index = train_matrix[u, :] > 0
        if len(candidate_item_index) < 1:
            print(f"item {i} is not found in the training set")
            pred_ratings.append(train_matrix[:, i].mean())
            continue

        this_item_sim = item_sim[i].copy()
        this_item_sim[~candidate_item_index] = 0
        candidate_sim = item_sim[i, candidate_item_index]

        topk_sim_items = np.abs(this_item_sim).argsort()[-k:][::-1]
        topk_sim = item_sim[i, topk_sim_items]
        topk_ratings = item_norm_train_matrix[u, topk_sim_items]
            
        item_ave = train_matrix[:,i].mean()
        topk_ave = np.sum((topk_ratings * topk_sim))/(topk_sim.sum()+1e-6)
        pred_rating = item_ave + topk_ave
        pred_ratings.append(pred_rating)
        if verbose:
            print(f"predict the rating {pred_rating:.4f} = user average {item_ave:.4f} + topk sim average {topk_ave:.4f}")
            print()
    return np.asarray(pred_ratings)

In [19]:
for k in [1, 2, 3, 4, 5, 10]:
    pred_ratings = item_cf_prediction(valid_users, valid_items, k)
    score = rmse(pred_ratings, valid_ratings)
    print(f"user CF with top-{k} similar items, RMSE = {score}")

user CF with top-1 similar users, RMSE = 1.5514636064968197
user CF with top-2 similar users, RMSE = 1.3931533204696869
user CF with top-3 similar users, RMSE = 1.3659986075499237
user CF with top-4 similar users, RMSE = 2.0621300994117795
user CF with top-5 similar users, RMSE = 1.3762175607962124
user CF with top-10 similar users, RMSE = 1.6793899948915336


## Matrix Factorization (Extension)

In [20]:
from scipy.linalg import svd
from scipy.sparse.linalg import svds

In [21]:
U, S, Vh = svd(train_matrix, full_matrices=False)
print(U.shape, S.shape, Vh.shape)

(4981, 4981) (4981,) (4981, 10845)


In [22]:
U, S, Vh = svds(train_matrix, k=10)
print(U.shape, S.shape, Vh.shape)

(4981, 10) (10,) (10, 10845)


In [23]:
def svd_prediction(valid_users, valid_items, k=10):
    pred = []
    U, S, Vh = svds(train_matrix, k)
    for u, i in zip(valid_users, valid_items):
        p = U[u,:] @ np.diag(S) @ Vh[:, i]
        pred.append(p)
    return np.asarray(pred)

In [24]:
for k in [5, 10, 50, 100]:
    pred_ratings = svd_prediction(valid_users, valid_items, k)
    score = rmse(pred_ratings, valid_ratings)
    print(f"SVD with {k} singular values, RMSE = {score}")

SVD with 5 singular values, RMSE = 3.920749421205831
SVD with 10 singular values, RMSE = 3.893203252399172
SVD with 50 singular values, RMSE = 3.867340713589129
SVD with 100 singular values, RMSE = 3.8810160966842435


## Matrix Factorization

In [25]:
def build_ncf_model(n_users, n_items, embed_size, output_layer='dot'):
    '''
    params:
        -n_users: number of user embedding vectors
        -n_items: number of item embedding vectors
        -embed_size: dimension of each embedding vector
        -output_layer: which instantiation of NCF to use ('dot' or 'mlp')

    return:
        a keras Model object for the constructed ncf model 
    '''

    # Get the users and items input
    user_input = Input(shape=(1,), dtype='int32', name='user_input')
    item_input = Input(shape=(1,), dtype='int32', name='item_input')

    
    # Get the embeddings of users and items
    
    user_emb = Embedding(output_dim=embed_size, input_dim=n_users, input_length=1)(user_input)
    user_emb = Reshape((embed_size,))(user_emb)

    item_emb = Embedding(output_dim=embed_size, input_dim=n_items, input_length=1)(item_input)
    item_emb = Reshape((embed_size,))(item_emb)

    if output_layer == 'dot':
        # Compute the dot product of users' and items' embeddings as the model output
        model_output = Dot(axes=1)([user_emb, item_emb])

    elif output_layer == 'mlp':

        # Concatenate the users' and items' embeddings as the input of MLP 
        mlp_input = Concatenate()([user_emb, item_emb])

        # First fully-connected layer
        dense_1 = Dense(512, activation='relu')(mlp_input)
        dense_1_dp = Dropout(0.15)(dense_1)

        # Second fully-connected layer
        dense_2 = Dense(512, activation='relu')(dense_1_dp)
        dense_2_dp = Dropout(0.15)(dense_2)

        # Final fully-connected layer to compute model output
        model_output = Dense(1)(dense_2_dp)
    else:
        raise NotImplementedError

    model = Model(inputs=[user_input, item_input],
                  outputs=model_output)
    return model
    
    

In [30]:
def case(embed_size=10, output_layer='dot', epochs=1):
    model = build_ncf_model(n_users, n_items, embed_size=embed_size, output_layer=output_layer)
    model.compile(optimizer='adam', loss='mse')
    history = model.fit(x=[train_users, train_items], y=train_ratings, epochs=epochs, verbose=1, callbacks=[ModelCheckpoint('model.h5')])
    y_pred = model.predict([train_users, train_items])
    print("TRAIN RMSE: ", rmse(y_pred, train_ratings))
    y_pred = model.predict([valid_users, valid_items])
    print("VALID RMSE: ", rmse(y_pred, valid_ratings))

In [31]:
case(embed_size=10, output_layer='dot', epochs=1)

3125/3125 [==============================] - 2s 601us/step - loss: 16.1138
TRAIN RMSE:  3.9317740612311214
VALID RMSE:  3.9449278138017005


In [32]:
case(embed_size=10, output_layer='dot', epochs=5)

Epoch 1/5
3125/3125 [==============================] - 2s 620us/step - loss: 16.1137
Epoch 2/5
3125/3125 [==============================] - 2s 610us/step - loss: 14.3278
Epoch 3/5
3125/3125 [==============================] - 2s 605us/step - loss: 8.7734
Epoch 4/5
3125/3125 [==============================] - 2s 605us/step - loss: 4.8003
Epoch 5/5
3125/3125 [==============================] - 2s 606us/step - loss: 2.8133
TRAIN RMSE:  1.389861713164762
VALID RMSE:  1.6435122699091151


In [33]:
case(embed_size=10, output_layer='dot', epochs=10)

Epoch 1/10
3125/3125 [==============================] - 2s 593us/step - loss: 16.1121
Epoch 2/10
3125/3125 [==============================] - 2s 603us/step - loss: 14.2756
Epoch 3/10
3125/3125 [==============================] - 2s 599us/step - loss: 8.7601
Epoch 4/10
3125/3125 [==============================] - 2s 604us/step - loss: 4.8530
Epoch 5/10
3125/3125 [==============================] - 2s 599us/step - loss: 2.9010
Epoch 6/10
3125/3125 [==============================] - 2s 603us/step - loss: 1.9412
Epoch 7/10
3125/3125 [==============================] - 2s 644us/step - loss: 1.4669
Epoch 8/10
3125/3125 [==============================] - 2s 657us/step - loss: 1.2248
Epoch 9/10
3125/3125 [==============================] - 2s 613us/step - loss: 1.0834
Epoch 10/10
3125/3125 [==============================] - 2s 604us/step - loss: 0.9837
TRAIN RMSE:  0.9381400280619244
VALID RMSE:  1.304390113031172


In [ ]:
case(embed_size=10, output_layer='mlp', epochs=1)

In [ ]:
case(embed_size=10, output_layer='mlp', epochs=5)

In [ ]:
case(embed_size=10, output_layer='mlp', epochs=10)